In [1]:
import pretty_midi
from corr_mat import calc_correlation, get_valid_loops
from track import Track
from util import get_instrument_type, create_loop_dict
import os
from tqdm import tqdm

In [2]:
test_file = "./test_midi.mid"
pm = pretty_midi.PrettyMIDI(test_file)

In [3]:
final_loops = []
for idx, instrument in enumerate(pm.instruments):
    instrument_type = get_instrument_type(instrument)
    track = Track(pm, instrument)
    note_list = track.notes
    lead_mat, lead_dur = calc_correlation(note_list)
    loops, loop_endpoints = get_valid_loops(track, lead_mat, lead_dur)
    print(f"found {len(loops)} loops in {instrument}")

    for endpoint in loop_endpoints:
        start, end, beats, density = endpoint
        time_sig = track.get_time_sig_at_time(start)
        print(idx, start, end, time_sig, beats, instrument_type, density)
    for loop in loops:
        final_loops.append(loop)
print(f"{len(final_loops)} total loops in {len(pm.instruments)} tracks")

found 1 loops in Instrument(program=0, is_drum=True, name="GOT TO HAVE YOUR LOVE     ")
0 18.113216000000005 22.641519999999996 (4, 4) 8.0 DRUM 3.875
found 1 loops in Instrument(program=39, is_drum=False, name="GOT TO HAVE YOUR LOVE     ")
1 9.056607999999999 13.584912000000005 (4, 4) 8.0 BASS 2.0
found 1 loops in Instrument(program=4, is_drum=False, name="GOT TO HAVE YOUR LOVE     ")
2 33.96227999999998 43.01888799999996 (4, 4) 16.0 PIANO 0.5625
found 1 loops in Instrument(program=48, is_drum=False, name="GOT TO HAVE YOUR LOVE     ")
3 24.905671999999992 29.433975999999983 (4, 4) 8.0 ENSEMBLE 0.75
found 1 loops in Instrument(program=48, is_drum=False, name="GOT TO HAVE YOUR LOVE     ")
4 24.905671999999992 29.433975999999983 (4, 4) 8.0 ENSEMBLE 0.75
found 1 loops in Instrument(program=28, is_drum=False, name="GOT TO HAVE YOUR LOVE     ")
5 27.169823999999988 31.69812799999998 (4, 4) 8.0 GUITAR 1.75
found 0 loops in Instrument(program=2, is_drum=False, name="GOT TO HAVE YOUR LOVE     "

In [4]:
def run_file(file_path, name):
    try:
        pm = pretty_midi.PrettyMIDI(file_path)
    except:
        print(f"failed to parse {file_path}, skipping")
        return 0,0,[]
    
    total_loops = 0
    loops = []
    for idx, instrument in enumerate(pm.instruments):
        instrument_type = get_instrument_type(instrument)
        track = Track(pm, instrument)
        note_list = track.notes
        lead_mat, lead_dur = calc_correlation(note_list)
        full_loops, loop_endpoints = get_valid_loops(track, lead_mat, lead_dur)
        for endpoint in loop_endpoints:
            time_sig = track.get_time_sig_at_time(endpoint[0])
            if time_sig is None:
                continue
            loop_dict = create_loop_dict(endpoint, idx, instrument_type, time_sig, name)
            loops.append(loop_dict)
        for loop_list in full_loops:
            if loop_list[0].duration != 0 or loop_list[-1].duration != 0:
                loop_list[0]
                print(name, loop_list[0], loop_list[-1])
        total_loops += len(loop_endpoints)
    return total_loops, len(pm.instruments), loops

In [5]:
full_directory = "D:\\Documents\\GigaMIDI\\Final_GigaMIDI_TISMIR\\Validatation-10%\\GigaMIDI-Val-Drum+Music-MD5"
len(os.listdir(full_directory))

25050

In [6]:
total_loops = 0
total_tracks = 0
num_files = 100
all_loops = []
for file in tqdm(os.listdir(full_directory)[:num_files]):
    full_path = os.path.join(full_directory, file)
    string_path = full_path
    num_loops, num_tracks, loops = run_file(full_path, string_path)
    total_loops += num_loops
    total_tracks += num_tracks
    for loop in loops:
        if len(loop) > 0:
            all_loops.append(loop)
print(f"Found {total_loops} loops in {total_tracks} tracks across {num_files} files")

  0%|          | 0/100 [00:00<?, ?it/s]

failed to parse D:\Documents\GigaMIDI\Final_GigaMIDI_TISMIR\Validatation-10%\GigaMIDI-Val-Drum+Music-MD5\.DS_Store, skipping


 20%|██        | 20/100 [00:09<00:24,  3.27it/s]d:\Documents\GigaMIDI\midi_loop_detection\.venv\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
100%|██████████| 100/100 [01:22<00:00,  1.21it/s]

Found 1441 loops in 929 tracks across 100 files
